# **STEP 2:** Patients Anonymization

In [1]:
#the python debugger is a library that provides an interactive source code debugger for Python programs
import pdb

import os
import os.path
import pydicom
import shutil
import zipfile
from zipfile import ZipFile
from glob import glob

### 2.1. Setting up the data and counting number of files to anonymize

In [2]:
rootPath = "/notebooks/disk2/RT_Dataset_2/01_2_Organized_Dataset/Patient_83/"
anonymizePath = "/notebooks/disk2/RT_Dataset_2/01_Anonymized_Patients/Patient_83/"

if rootPath.split("/")[5].startswith("_"):
    patient_number = (rootPath.split("/")[5]).split("_")[2]
else:
    patient_number = (rootPath.split("/")[5]).split("_")[1]

In [3]:
# Calculate the number of DICOM files inside paciente_N folder
totalFiles2 = glob(rootPath + "*/*")
print("Total of %d DICOM images." %len(totalFiles2))

Total of 24 DICOM images.


### 2.2. Anonymizing all folders of DICOM image files of a Patient N

In [4]:
# Functin to copy all files from one directory to another
def copyFiles(rootPath, anonymizePath):
    if not os.path.exists(anonymizePath): 
        shutil.copytree(rootPath, anonymizePath)
        print("--> Anonymized directory created")
    else:
        print("--> Anonymized directory " + anonymizePath.split("/")[-2] + " already exists.")
        alltoAnonymize = glob(anonymizePath + '*/*')
        print("--> Total of %d DICOM images." %len(alltoAnonymize))

copyFiles(rootPath, anonymizePath)

--> Anonymized directory created


In [5]:
# Anonymize patient data by setting a null value to the corresponding tags
def getanonfiles(anonymizePath):
    name = (0x0010, 0x0010)
    deletephi = [
                (0x0008, 0x0050), # Accession Number
                (0x0008, 0x0080), # Institution Name
                (0x0008, 0x0090), # Refering Physicion's Name
                (0x0008, 0x1010), # Station Name
                (0x0008, 0x1070), # Operator's Name
                (0x0010, 0x0020), # Patient's ID
                (0x0010, 0x0030), # Patient's Birth Date 
                (0x0010, 0x1000), #Other Patient IDs 
                (0x0010, 0x1001), #Other Patient Names   
                #(0x0010, 0x1010), # Patient's Age
                (0x0010, 0x1020), # Patient's Size
                (0x0010, 0x1030), # Patient's Weight
                (0x0018, 0x1000), # Device Serial Number
                (0x0018, 0x1030), # Protocol Name
                (0x0020, 0x0010) # Study ID
                ]
    for directorypath, directorynames, filenames in os.walk(anonymizePath):
        for file in filenames:
            selectFile = os.path.join(directorypath, file)
            readFile = pydicom.read_file(selectFile, force=True)

            if name in readFile:
                readFile[name].value = f'P_{patient_number}'
            
            for tag in deletephi:
                if tag in readFile:
                    readFile[tag].value = ''
            
            pydicom.filewriter.write_file(selectFile, readFile)
    print('--> Task completed.')       
            
getanonfiles(anonymizePath)

/opt/conda/lib/python3.10/site-packages/pydicom/filereader.py:310: UserWarning: Expected implicit VR, but found explicit VR - using explicit VR for reading
  warnings.warn(message, UserWarning)


--> Task completed.


In [6]:
# Delete patients names from Structures file
for folder in os.listdir(anonymizePath):
    subfolder = os.path.join(anonymizePath, folder)
    if os.path.isdir(subfolder):  # Check if it's a directory
        for file in os.listdir(subfolder):
            if "CT_Images" in os.path.basename(file):
                if "DCMRT_Structures" in file:
                    oldpath = os.path.join(subfolder, file)
                    newpath = os.path.join(subfolder, "DCMRT_Structures.dcm")
                    os.rename(oldpath, newpath)
                    print(f"File {file} renamed to {newpath}")
                elif "RTSS" in file:
                    oldpath = os.path.join(subfolder, file)
                    newpath = os.path.join(subfolder, "RTSS.dcm")
                    os.rename(oldpath, newpath)
                    print(f"File {file} renamed to {newpath}")


### 2.3. Zip

In [7]:
folderPath = "/notebooks/disk2/RT_Dataset_2/01_Anonymized_Patients/Patient_83/"
outputPath = "/notebooks/disk2/RT_Dataset_2/04_Zipped_Directories/02_Anonymized_Dataset/Patient_83.zip"

In [8]:
# Zip the anonymized directory
def zip_folder(folderPath, outputPath):
    with zipfile.ZipFile(outputPath, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(folderPath):
            for file in files:
                zipf.write(os.path.join(root, file))
    print("--> Folder Zipped")
zip_folder(folderPath, outputPath)

--> Folder Zipped
